In [ ]:
!pip install spotipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.8/279.8 kB 7.1 MB/s eta 0:00:00


In [ ]:
!pip install tensorflow

In [ ]:
client_id = "5c49487b3d084f6184994c78b52d96ef"
client_secret = "f02ac9d386fd489abefc40c2b850256f"
redirect_uri = "https://example.com:3000"

In [ ]:
import pandas as pd
import numpy as np
import json
import re
import sys
import itertools
import cv2
from PIL import Image

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from skimage import io
import matplotlib.pyplot as plt

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util

import warnings
warnings.filterwarnings("ignore")

In [ ]:
%matplotlib inline

In [ ]:
#another useful command to make data exploration easier
# NOTE: if you are using a massive dataset, this could slow down your code.
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows", None)

In [ ]:
df = pd.read_csv('kaggleMusicMoodFinal.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'kaggleMusicMoodFinal.csv'

In [ ]:
spotify_df = df.copy()

In [ ]:
spotify_df.shape

(22818, 27)

In [ ]:
float_cols = spotify_df.dtypes[spotify_df.dtypes == 'float64'].index.values

In [ ]:
ohe_cols = 'popularity'

In [ ]:
# create 5 point buckets for popularity
spotify_df['bucket_popularity'] = spotify_df['popularity'].apply(lambda x: int(x/5))

In [ ]:
spotify_df['consolidates_genre_lists_upd'] = spotify_df['consolidates_genre_lists'].apply(
    lambda x: [re.sub(' ','_', i) for i in re.findall(r"'([^']*)'", x)] if isinstance(x, str) else []
)


In [ ]:
spotify_df.head()

,Unnamed: 0.1,Unnamed: 0,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,artists_upd_v1,artists_upd_v2,artists_upd,artists_song,consolidates_genre_lists,Mood,bucket_popularity,consolidates_genre_lists_upd
0,0,0,0.177,1989,0.568,['조정현'],0.447,237688,0.2150,0,2ghebdwe2pNXT4eL34T7pW,0.000001,10,0.0649,-16.478,1,그아픔까지사랑한거야,31,1989-06-15,0.0272,71.979,['조정현'],[],['조정현'],조정현그아픔까지사랑한거야,['classic_korean_pop'],Sad,6,[classic_korean_pop]
1,1,1,0.352,1992,0.381,['黑豹'],0.353,316160,0.6860,0,3KIuCzckjdeeVuswPo20mC,0.000000,11,0.0568,-9.103,1,DON'T BREAK MY HEART,35,1992-12-22,0.0395,200.341,['黑豹'],[],['黑豹'],黑豹DON'T BREAK MY HEART,"['chinese_indie', 'chinese_indie_rock']",Sad,7,"[chinese_indie, chinese_indie_rock]"
2,2,2,0.458,1963,0.987,['黃國隆'],0.241,193480,0.0437,0,4prhqrLXYMjHJ6vpRAlasx,0.000453,5,0.2650,-24.571,1,藝旦調,23,1963-05-28,0.0443,85.936,['黃國隆'],[],['黃國隆'],黃國隆藝旦調,[],Sad,4,[]
3,3,3,0.796,1963,0.852,"['黃國隆', '王秋玉']",0.711,145720,0.1110,0,5xFXTvnEe03SyvFpo6pEaE,0.000000,2,0.0695,-20.741,0,草螟弄雞公,23,1963-05-28,0.0697,124.273,"['黃國隆', '王秋玉']",[],"['黃國隆', '王秋玉']",黃國隆草螟弄雞公,[],Sad,4,[]
4,4,4,0.704,1963,0.771,['黃國隆'],0.610,208760,0.1750,0,6Pqs2suXEqCGx7Lxg5dlrB,0.000000,7,0.0309,-20.232,1,思想起,23,1963-05-28,0.0419,124.662,['黃國隆'],[],['黃國隆'],黃國隆思想起,[],Sad,4,[]


In [ ]:
#simple function to create OHE features
#this gets passed later on
def One_Hot_Encode_Prep(df, column, new_name):
    """
    Create One Hot Encoded features of a specific column

    Parameters:
        df (pandas dataframe): Spotify Dataframe
        column (str): Column to be processed
        new_name (str): new column name to be used

    Returns:
        tf_df: One hot encoded features
    """

    tf_df = pd.get_dummies(df[column])
    feature_names = tf_df.columns
    tf_df.columns = [new_name + "|" + str(i) for i in feature_names]
    tf_df.reset_index(drop = True, inplace = True)
    return tf_df



#function to build entire feature set
def Feature_Set_Using_TDiDF(df, float_cols):
    """
    Process spotify df to create a final set of features that will be used to generate recommendations

    Parameters:
        df (pandas dataframe): Spotify Dataframe
        float_cols (list(str)): List of float columns that will be scaled

    Returns:
        final: final set of features
    """

    #tfidf genre lists
    tfidf = TfidfVectorizer()
    tfidf_matrix =  tfidf.fit_transform(df['consolidates_genre_lists_upd'].apply(lambda x: " ".join(x)))
    genre_df = pd.DataFrame(tfidf_matrix.toarray())
    genre_df.columns = ['genre' + "|" + i for i in tfidf.get_feature_names_out()]
    genre_df.reset_index(drop = True, inplace=True)

    #explicity_ohe = One_Hot_Encode_Prep(df, 'explicit','exp')
    year_ohe = One_Hot_Encode_Prep(df, 'year','year') * 0.5
    popularity_ohe = One_Hot_Encode_Prep(df, 'bucket_popularity','pop') * 0.15

    #scale float columns
    floats = df[float_cols].reset_index(drop = True)
    scaler = MinMaxScaler()
    floats_scaled = pd.DataFrame(scaler.fit_transform(floats), columns = floats.columns) * 0.2

    #concanenate all features
    final = pd.concat([genre_df, floats_scaled, popularity_ohe, year_ohe], axis = 1)

    #add song id
    final['id']=df['id'].values

    return final


def Get_Spotify_Playlist_DataFrame(playlist_name,id_dic, df):
    """
    Pull songs from a specific playlist.

    Parameters:
        playlist_name (str): name of the playlist you'd like to pull from the spotify API
        id_dic (dic): dictionary that maps playlist_name to playlist_id
        df (pandas dataframe): spotify datafram

    Returns:
        playlist: all songs in the playlist THAT ARE AVAILABLE IN THE KAGGLE DATASET
    """

    #generate playlist dataframe
    playlist = pd.DataFrame()
    playlist_name = playlist_name

    #print(len(sp.playlist(id_dic[playlist_name])['tracks']['items']))

    for ix, i in enumerate(sp.playlist(id_dic[playlist_name])['tracks']['items']):
        #print(i['track']['artists'][0]['name'])
        playlist.loc[ix, 'artist'] = i['track']['artists'][0]['name']
        playlist.loc[ix, 'name'] = i['track']['name']
        playlist.loc[ix, 'id'] = i['track']['id'] # ['uri'].split(':')[2]
        playlist.loc[ix, 'url'] = i['track']['album']['images'][1]['url']
        playlist.loc[ix, 'date_added'] = i['added_at']

    playlist['date_added'] = pd.to_datetime(playlist['date_added'])

    #playlist = playlist[playlist['id'].isin(df['id'].values)].sort_values('date_added',ascending = False)

    return playlist




def visualize_songs(df):
    """
    Visualize cover art of the songs in the inputted dataframe

    Parameters:
        df (pandas dataframe): Playlist Dataframe
    """

    temp = df['url'].values
    plt.figure(figsize=(15,int(0.625 * len(temp))))
    columns = 5

    for i, url in enumerate(temp):
        plt.subplot(len(temp) / columns + 1, columns, i + 1)

        image = io.imread(url)
        plt.imshow(image)
        plt.xticks(color = 'w', fontsize = 0.1)
        plt.yticks(color = 'w', fontsize = 0.1)
        plt.xlabel(df['name'].values[i], fontsize = 12)
        plt.tight_layout(h_pad=0.4, w_pad=0)
        plt.subplots_adjust(wspace=None, hspace=None)

    plt.show()




def generate_playlist_feature(complete_feature_set, playlist_df, weight_factor):
    """
    Summarize a user's playlist into a single vector

    Parameters:
        complete_feature_set (pandas data00frame): Dataframe which includes all of the features for the spotify songs
        playlist_df (pandas dataframe): playlist dataframe
        weight_factor (float): float value that represents the recency bias. The larger the recency bias, the most priority recent songs get. Value should be close to 1.

    Returns:
        playlist_feature_set_weighted_final (pandas series): single feature that summarizes the playlist
        playlist_features_not_in_dataframe (pandas dataframe):
    """

    playlist_features_in_dataframe = complete_feature_set[complete_feature_set['id'].isin(playlist_df['id'].values)]#.drop('id', axis = 1).mean(axis =0)
    playlist_features_in_dataframe = playlist_features_in_dataframe.merge(playlist_df[['id','date_added']], on = 'id', how = 'inner')
    playlist_features_not_in_dataframe = complete_feature_set[~complete_feature_set['id'].isin(playlist_df['id'].values)]#.drop('id', axis = 1)
    print("Matched tracks with Kaggle dataset:", len(playlist_features_in_dataframe))

    playlist_feature_set = playlist_features_in_dataframe.sort_values('date_added',ascending=False)

    most_recent_date = playlist_feature_set.iloc[0,-1]

    for ix, row in playlist_feature_set.iterrows():
        playlist_feature_set.loc[ix,'months_from_recent'] = int((most_recent_date.to_pydatetime() - row.iloc[-1].to_pydatetime()).days / 30)

    playlist_feature_set['weight'] = playlist_feature_set['months_from_recent'].apply(lambda x: weight_factor ** (-x))

    playlist_feature_set_weighted = playlist_feature_set.copy()
    #print(playlist_feature_set_weighted.iloc[:,:-4].columns)
    playlist_feature_set_weighted.update(playlist_feature_set_weighted.iloc[:,:-4].mul(playlist_feature_set_weighted.weight,0))
    playlist_feature_set_weighted_final = playlist_feature_set_weighted.iloc[:, :-4]
    #playlist_feature_set_weighted_final['id'] = playlist_feature_set['id']

    return playlist_feature_set_weighted_final.sum(axis = 0), playlist_features_not_in_dataframe





def Recommend_Playlist(df, features, nonplaylist_features):
    """
    Pull songs from a specific playlist.

    Parameters:
        df (pandas dataframe): spotify dataframe
        features (pandas series): summarized playlist feature
        nonplaylist_features (pandas dataframe): feature set of songs that are not in the selected playlist

    Returns:
        non_playlist_df_top_40: Top 40 recommendations for that playlist
    """

    non_playlist_df = df[df['id'].isin(nonplaylist_features['id'].values)]
    non_playlist_df['sim'] = cosine_similarity(nonplaylist_features.drop('id', axis = 1).values, features.values.reshape(1, -1))[:,0]
    non_playlist_df_top_40 = non_playlist_df.sort_values('sim',ascending = False).head(40)
    non_playlist_df_top_40['url'] = non_playlist_df_top_40['id'].apply(lambda x: sp.track(x)['album']['images'][1]['url'])

    return non_playlist_df_top_40



In [ ]:
scope = 'user-library-read'

if len(sys.argv) > 1:
    username = sys.argv[1]
else:
    print("Usage: %s username" % (sys.argv[0],))
    sys.exit()

In [ ]:
from spotipy.oauth2 import SpotifyOAuth

sp_oauth = SpotifyOAuth(
    client_id=client_id,
    client_secret=client_secret,
    redirect_uri='https://example.com:3000',  # must match Spotify dashboard
    scope=scope
)

auth_url = sp_oauth.get_authorize_url()
print("Open this URL in your browser:")
print(auth_url)

Open this URL in your browser:
https://accounts.spotify.com/authorize?client_id=5c49487b3d084f6184994c78b52d96ef&response_type=code&redirect_uri=https%3A%2F%2Fexample.com%3A3000&scope=user-library-read


In [ ]:
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [ ]:
sp = spotipy.Spotify(auth=sp_oauth)

In [ ]:
# After you open the URL and login, Spotify redirects you to:
# http://127.0.0.1:8090/callback?code=AUTH_CODE
# Copy that full redirect URL and paste it here:
response_url = input("Enter the full redirect URL: ")

# Fetch access token
code = sp_oauth.parse_response_code(response_url)
token_info = sp_oauth.get_access_token(code)

# Create Spotify client with this token
from spotipy import Spotify
sp = Spotify(auth=token_info['access_token'])

# Now you can call user-specific endpoints
results = sp.current_user_playlists()
for playlist in results['items']:
    print(playlist['name'])


Enter the full redirect URL: http://127.0.0.1:8090/callback?code=AQCP1rusPq3i2NoU1x6lhTaFKhkLfp_xRgV4905hEtRU5p4ud6lPLg8vTajCQcqEcDl6VEpdUt7Fkq081EYQLMV9YkrK_Dw0PG5TG4IFJkjf138Jy3iLdDupGO3eSeclDDWkdNX5tm_AXNvveOkokv5SDsRRFwJ65AJ1Vv6VvpXjzPqxSCIjlieRnTLBbtxn8Nvtxyg
Hehe
Sjsj
Yo
heyy


In [ ]:
#gather playlist names and images.
#images aren't going to be used until I start building a UI
id_name = {}
list_photo = {}
for i in sp.current_user_playlists()['items']:

    id_name[i['name']] = i['uri'].split(':')[2]
    list_photo[i['uri'].split(':')[2]] = i['images'][0]['url']

In [ ]:
print(id_name)

{'Hehe': '39rWDQJdpu2eW3zamzUbsO', 'Sjsj': '2jSoeGOT5UOWheiHGFVnz0', 'Yo': '6kNnPvI9hedEDyNbJApstX', 'heyy': '7wILD4m6ggzMSfF3jWGoRv'}


In [ ]:
def ChooseDataset(x):
    if x == "Disgust":
        return spotify_df[spotify_df['Mood'].isin(['Energetic', 'Happy', 'Calm'])]
    if x == "Angry":
        return spotify_df[spotify_df['Mood'].isin(['Energetic', 'Calm'])]
    if x == "Fear":
        return spotify_df[spotify_df['Mood'].isin(['Happy', 'Calm'])]
    if x == "Happy":
        return spotify_df[spotify_df['Mood'].isin(['Sad', 'Happy', 'Calm'])]
    if x == "Sad":
        return spotify_df[spotify_df['Mood'].isin(['Sad', 'Happy', 'Calm'])]
    if x == "Surprise":
        return spotify_df[spotify_df['Mood'].isin(['Energetic', 'Happy', 'Sad'])]
    return spotify_df

In [ ]:
def Recommend_Top40(x):
    #..............................
    #.............................
    #............................

    O_df = ChooseDataset(x)

    # Feature Engineering from main dataframe
    complete_feature_set = Feature_Set_Using_TDiDF(O_df, float_cols=float_cols)#.mean(axis = 0)

    # collecting spotify user playlist dataframe
    one_playlist_from_spotify = Get_Spotify_Playlist_DataFrame('Yo', id_name, O_df)
    print(one_playlist_from_spotify.head())
    print("Number of tracks from Spotify playlist:", len(one_playlist_from_spotify))


    # linear vector for recommendation
    features_in_the_playlist, features_not_in_the_playlist = generate_playlist_feature(complete_feature_set, one_playlist_from_spotify, 1.09)


    # Recommended Songs store here
    top40_recommendation = Recommend_Playlist(spotify_df, features_in_the_playlist, features_not_in_the_playlist)


    visualize_songs(top40_recommendation)




In [ ]:
from tensorflow.keras.models import load_model

model = load_model('MoodDetector.h5', compile = True)

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'MoodDetector.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
import cv2

# Load Haar cascade from OpenCV’s built-in data
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")


In [ ]:
def moodNamePrintFromLabel(n):
    if n == 0: result = "Angry"
    elif n == 1: result = "Disgust"
    elif n == 2: result = "Fear"
    elif n == 3: result = "Happy"
    elif n == 4: result = "Sad"
    elif n == 5: result = "Surprise"
    elif n == 6: result = "Neutral"

    return result

def imageToMoodDetection(img_path):

    img = cv2.imread(img_path)

    Gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(Gray_img, 1.3, 3)

    for (x,y,w,h) in faces:
      pxl_lst = []
      for i in range(y, y+h):
        lst = []
        for j in range(x, x+w):
          lst.append(Gray_img[i][j])
        pxl_lst.append(lst)
      single_face = np.array(pxl_lst)
      single_face = np.reshape(single_face,(h,w,))

      resized_img = cv2.resize(single_face, (48,48), interpolation = cv2.INTER_AREA)

      resized_img = np.reshape(resized_img,(1,48,48,1))/255.0

      #passing to model
      result = np.argmax(model.predict(resized_img), axis=-1)
      if result is not None:
        print(moodNamePrintFromLabel(result))

In [ ]:
import cv2
from tensorflow.keras.models import load_model
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Load your trained model (adjust path if needed)
model = load_model('MoodDetector.h5')

# Spotify setup (use your credentials)
client_id = 'c96ea1e490554b5eaa3bc0c974711bb7'
client_secret = 'fbcd069b9648412089ff7fcc4731f172'
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

# Haar Cascade for face detection (download from OpenCV GitHub if not in your folder)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Function to predict mood from a face image
def predict_mood(face_img):
    face_img = cv2.resize(face_img, (48, 48))  # Resize to match model input
    face_img = face_img / 255.0  # Normalize
    face_img = np.expand_dims(face_img, axis=0)  # Add batch dimension
    face_img = np.expand_dims(face_img, axis=-1)  # For grayscale
    prediction = model.predict(face_img)
    moods = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']  # Match your model's classes
    return moods[np.argmax(prediction)]

# Function to recommend songs based on mood
def recommend_songs(mood, num_songs=5):
    query = f'mood:{mood.lower()}'  # Simple query; adjust for better results
    results = sp.search(q=query, type='track', limit=num_songs)
    tracks = results['tracks']['items']
    recommendations = [track['name'] + ' by ' + track['artists'][0]['name'] for track in tracks]
    return recommendations

# Live webcam capture
cap = cv2.VideoCapture(0)  # 0 for default webcam

if not cap.isOpened():
    print("Error: Could not open webcam.")
else:
    print("Press 'q' to quit and get recommendations.")
    detected_mood = None
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        for (x, y, w, h) in faces:
            face = gray[y:y+h, x:x+w]
            detected_mood = predict_mood(face)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            cv2.putText(frame, detected_mood, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX,
                        0.9, (0, 255, 0), 2)

        cv2.imshow('Live Emotion Detection', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            # Save the frame before breaking
            cv2.imwrite("captured_frame.jpg", frame)
            print("Image saved as captured_frame.jpg")
            break

    cap.release()
    cv2.destroyAllWindows()

    if detected_mood:
        print(f"Detected Mood: {detected_mood}")
        songs = recommend_songs(detected_mood)
        print("Recommended Songs:")
        for song in songs:
            print(song)
    else:
        print("No face detected.")


In [ ]:
Image_path = 'captured_frame.jpg'

# Recommended songs for 'EDM' Playlist for Neutral Mood

In [ ]:
Recommend_Top40(imageToMoodDetection(Image_path))

In [ ]:
Recommend_Top40('Happy')

          artist            name                      id  \
0   Armaan Malik         Jab Tak  35RWttZt8zGGzT0mWtlXpn   
1   Armaan Malik  Bol Do Na Zara  5XD2tP1DRH2NJgp45HiEQA   
2  Darshan Raval       Asal Mein  0rMeOAvbQZ2RTG4B6L1M4p   
3   Arijit Singh         Humdard  1OEmd2iXU8pWG7CrKNpGUg   
4  One Direction   Night Changes  5O2P9iiztwhomNh8xkR9lJ   

                                                 url                date_added  
0  https://i.scdn.co/image/ab67616d00001e0202dde7... 2024-09-15 10:15:29+00:00  
1  https://i.scdn.co/image/ab67616d00001e023f334d... 2024-09-15 10:15:37+00:00  
2  https://i.scdn.co/image/ab67616d00001e026597f0... 2024-10-13 15:07:04+00:00  
3  https://i.scdn.co/image/ab67616d00001e02a2ef89... 2024-10-13 15:07:10+00:00  
4  https://i.scdn.co/image/ab67616d00001e02d304ba... 2024-10-13 15:07:21+00:00  
Number of tracks from Spotify playlist: 34
Matched tracks with Kaggle dataset: 0


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
def ChooseDataset(mood):
    """
    Map mood to Spotify search parameters
    """
    mood_to_search = {
        "Disgust": "rock metal angry intense",
        "Angry": "metal hard rock aggressive",
        "Fear": "ambient dark atmospheric scary",
        "Happy": "pop dance upbeat happy",
        "Sad": "sad acoustic emotional melancholic",
        "Surprise": "electronic experimental surprising",
        "Calm": "chill ambient peaceful calm",
        "Energetic": "edm energetic workout pump-up"
    }
    return mood_to_search.get(mood, "pop")


def Search_Spotify_By_Mood_Simple(mood_query, limit=40):
    """
    Simple search using mood keywords without audio features
    """
    try:
        # Search for tracks based on mood keywords
        results = sp.search(q=mood_query, type='track', limit=limit)

        recommendations = []
        for track in results['tracks']['items']:
            recommendations.append({
                'id': track['id'],
                'name': track['name'],
                'artist': track['artists'][0]['name'],
                'album': track['album']['name'],
                'url': track['album']['images'][1]['url'] if track['album']['images'] and len(track['album']['images']) > 1 else None,
                'preview_url': track['preview_url'],
                'external_url': track['external_urls']['spotify'],
                'popularity': track['popularity']
            })

        return pd.DataFrame(recommendations)

    except Exception as e:
        print(f"Error in simple search: {e}")
        return pd.DataFrame()


def Search_Spotify_By_Genre(genre, limit=40):
    """
    Search by genre only
    """
    try:
        results = sp.search(q=f'genre:{genre}', type='track', limit=limit)

        recommendations = []
        for track in results['tracks']['items']:
            recommendations.append({
                'id': track['id'],
                'name': track['name'],
                'artist': track['artists'][0]['name'],
                'url': track['album']['images'][1]['url'] if track['album']['images'] and len(track['album']['images']) > 1 else None,
                'popularity': track['popularity']
            })

        return pd.DataFrame(recommendations)

    except Exception as e:
        print(f"Error searching by genre: {e}")
        return pd.DataFrame()


def Recommend_Top40_By_Mood_Simple(mood):
    """
    Simple mood-based recommendation without audio features
    """
    print(f"\n{'='*50}")
    print(f"Searching for {mood} mood...")
    print(f"{'='*50}")

    # Get mood search query
    mood_query = ChooseDataset(mood)
    print(f"Search query: {mood_query}")

    # Search Spotify
    recommendations = Search_Spotify_By_Mood_Simple(mood_query, limit=40)

    if not recommendations.empty:
        print(f"🎵 Found {len(recommendations)} recommendations for {mood} mood!")

        # Sort by popularity
        recommendations = recommendations.sort_values('popularity', ascending=False)

        # Display top recommendations
        print(f"\n=== Top 10 {mood} Mood Recommendations ===")
        for i, (_, row) in enumerate(recommendations.head(10).iterrows(), 1):
            print(f"{i}. {row['name']} - {row['artist']} (Popularity: {row['popularity']})")

        return recommendations
    else:
        print("❌ No recommendations found.")
        return pd.DataFrame()


# Alternative: Use Spotify's curated playlists for moods
def Get_Mood_Playlists(mood):
    """
    Get mood-based curated playlists
    """
    mood_playlists = {
        "Happy": "spotify:playlist:37i9dQZF1DXdPec7aLTmlC",  # Happy Hits
        "Sad": "spotify:playlist:37i9dQZF1DX7qK8ma5wgG1",   # Sad Songs
        "Angry": "spotify:playlist:37i9dQZF1DX4sWSpwq3LiO", # Rock Classics
        "Calm": "spotify:playlist:37i9dQZF1DX4WYpdgoIcn6",  # Chill Hits
        "Energetic": "spotify:playlist:37i9dQZF1DX0XUsuxWHRQd", # RapCaviar
        "Workout": "spotify:playlist:37i9dQZF1DX76Wlfdnj7AP" # Beast Mode
    }

    playlist_id = mood_playlists.get(mood)
    if playlist_id:
        try:
            playlist = sp.playlist(playlist_id)
            tracks = sp.playlist_tracks(playlist_id, limit=40)

            recommendations = []
            for item in tracks['items']:
                track = item['track']
                if track:
                    recommendations.append({
                        'id': track['id'],
                        'name': track['name'],
                        'artist': track['artists'][0]['name'],
                        'url': track['album']['images'][1]['url'] if track['album']['images'] and len(track['album']['images']) > 1 else None,
                        'popularity': track['popularity']
                    })

            return pd.DataFrame(recommendations)
        except Exception as e:
            print(f"Error getting playlist: {e}")

    return pd.DataFrame()


# Test function
def test_simple_recommendations():
    """Test the simple recommendation system"""
    moods = ["Happy", "Sad", "Angry", "Calm", "Energetic"]

    for mood in moods:
        try:
            # Method 1: Simple search
            print(f"\n--- Method 1: Keyword Search for {mood} ---")
            recommendations = Recommend_Top40_By_Mood_Simple(mood)

            if recommendations.empty:
                # Method 2: Curated playlists
                print(f"\n--- Method 2: Curated Playlist for {mood} ---")
                recommendations = Get_Mood_Playlists(mood)

            if not recommendations.empty:
                print(f"✅ Successfully found {len(recommendations)} songs for {mood} mood!")
                if len(recommendations) > 0:
                    visualize_songs(recommendations.head(10))
            else:
                print(f"❌ Failed to find songs for {mood} mood")

        except Exception as e:
            print(f"💥 Error processing {mood} mood: {e}")

        print("\n" + "-"*50 + "\n")


# Run the simple test
test_simple_recommendations()




--- Method 1: Keyword Search for Happy ---

Searching for Happy mood...
Search query: pop dance upbeat happy
🎵 Found 40 recommendations for Happy mood!

=== Top 10 Happy Mood Recommendations ===
1. Hush - BYRNE (Popularity: 38)
2. Feel Good - Saint Motel (Popularity: 37)
3. The Wrong Party (with Fitz and The Tantrums) - Andy Grammer (Popularity: 35)
4. Energetic Upbeat Pop - Morninglightmusic (Popularity: 35)
5. Holding Out - eleventyseven (Popularity: 35)
6. Uplifting Upbeat Energetic Instrumental - Romansenykmusic (Popularity: 35)
7. Easy Thing - Dom Fera (Popularity: 34)
8. Upbeat Energetic Fun Happy Kids - Gold-Tiger (Popularity: 33)
9. Happy Inspiring Upbeat - Romansenykmusic (Popularity: 27)
10. Ciao - Boy In Space (Popularity: 25)
✅ Successfully found 40 songs for Happy mood!
💥 Error processing Happy mood: Number of rows must be a positive integer, not 3.0

--------------------------------------------------


--- Method 1: Keyword Search for Sad ---

Searching for Sad mood...
S

True

<Figure size 1500x600 with 0 Axes>

<Figure size 1500x600 with 0 Axes>

<Figure size 1500x600 with 0 Axes>

<Figure size 1500x600 with 0 Axes>

<Figure size 1500x600 with 0 Axes>

In [ ]:
def ChooseDataset(mood):
    """
    Map mood to Spotify search parameters with Indian context
    """
    mood_to_search = {
        "Disgust": "rock metal angry intense bollywood",
        "Angry": "metal hard rock aggressive hindi rap",
        "Fear": "ambient dark atmospheric scary bollywood thriller",
        "Happy": "bollywood dance upbeat happy hindi punjabi",
        "Sad": "sad acoustic emotional melancholic hindi bollywood",
        "Surprise": "fusion experimental surprising indian electronic",
        "Calm": "chill ambient peaceful calm indian classical",
        "Energetic": "bollywood dance energetic punjabi bhangra item song"
    }
    return mood_to_search.get(mood, "bollywood hindi")


def Search_Spotify_By_Mood_India(mood_query, limit=40):
    """
    Search for tracks based on mood keywords with India market focus
    """
    try:
        # Search for tracks based on mood keywords in Indian market
        results = sp.search(q=mood_query, type='track', limit=limit, market='IN')

        recommendations = []
        for track in results['tracks']['items']:
            recommendations.append({
                'id': track['id'],
                'name': track['name'],
                'artist': track['artists'][0]['name'],
                'album': track['album']['name'],
                'url': track['album']['images'][1]['url'] if track['album']['images'] and len(track['album']['images']) > 1 else None,
                'preview_url': track['preview_url'],
                'external_url': track['external_urls']['spotify'],
                'popularity': track['popularity']
            })

        return pd.DataFrame(recommendations)

    except Exception as e:
        print(f"Error in India-specific search: {e}")
        return pd.DataFrame()


def Search_Spotify_By_Genre_India(genre, limit=40):
    """
    Search by genre only in Indian market
    """
    try:
        # Add Indian genres and context
        indian_genres = {
            'bollywood': 'bollywood hindi',
            'classical': 'indian classical hindustani carnatic',
            'folk': 'indian folk regional',
            'devotional': 'devotional bhajan kirtan',
            'punjabi': 'punjabi bhangra',
            'tamil': 'tamil kollywood',
            'telugu': 'telugu tollywood',
            'regional': 'regional indian languages'
        }

        search_query = indian_genres.get(genre.lower(), f'genre:{genre}')
        results = sp.search(q=search_query, type='track', limit=limit, market='IN')

        recommendations = []
        for track in results['tracks']['items']:
            recommendations.append({
                'id': track['id'],
                'name': track['name'],
                'artist': track['artists'][0]['name'],
                'url': track['album']['images'][1]['url'] if track['album']['images'] and len(track['album']['images']) > 1 else None,
                'popularity': track['popularity']
            })

        return pd.DataFrame(recommendations)

    except Exception as e:
        print(f"Error searching by genre in India: {e}")
        return pd.DataFrame()


def Get_Indian_Top_Charts():
    """
    Get current top charts in India across different categories
    """
    try:
        # Get India's top 50 playlist
        india_top_50 = sp.playlist('37i9dQZEVXbLZ52XmnySJg')  # Top 50 India
        tracks = sp.playlist_tracks('37i9dQZEVXbLZ52XmnySJg', limit=50, market='IN')

        recommendations = []
        for item in tracks['items']:
            track = item['track']
            if track:
                recommendations.append({
                    'id': track['id'],
                    'name': track['name'],
                    'artist': track['artists'][0]['name'],
                    'url': track['album']['images'][1]['url'] if track['album']['images'] and len(track['album']['images']) > 1 else None,
                    'popularity': track['popularity'],
                    'chart_position': len(recommendations) + 1
                })

        return pd.DataFrame(recommendations)
    except Exception as e:
        print(f"Error getting Indian top charts: {e}")
        return pd.DataFrame()


def Recommend_Top40_By_Mood_India(mood):
    """
    Mood-based recommendation focusing on Indian popular music
    """
    print(f"\n{'='*50}")
    print(f"Searching for {mood} mood songs popular in India...")
    print(f"{'='*50}")

    # Get mood search query with Indian context
    mood_query = ChooseDataset(mood)
    print(f"Search query: {mood_query}")

    # Method 1: Search by mood with Indian market
    recommendations = Search_Spotify_By_Mood_India(mood_query, limit=40)

    if recommendations.empty:
        print("No mood-specific results found. Trying Indian playlists...")
        # Method 2: Get from Indian playlists
        recommendations = Get_Mood_Playlists_India(mood)

    if not recommendations.empty:
        print(f"🎵 Found {len(recommendations)} recommendations for {mood} mood in India!")

        # Sort by popularity
        recommendations = recommendations.sort_values('popularity', ascending=False)

        # Display top recommendations
        print(f"\n=== Top 10 {mood} Mood Recommendations (Popular in India) ===")
        for i, (_, row) in enumerate(recommendations.head(10).iterrows(), 1):
            print(f"{i}. {row['name']} - {row['artist']} (Popularity: {row['popularity']})")

        return recommendations
    else:
        print("❌ No recommendations found for India.")
        return pd.DataFrame()


def Get_Mood_Playlists_India(mood):
    """
    Get mood-based curated playlists popular in India
    """
    # Indian mood-specific playlists
    mood_playlists_india = {
        "Happy": [
            "37i9dQZF1DX0XHGtC6tta5",  # Bollywood Central
            "37i9dQZF1DX1spT6G94GFC",  # Happy Bollywood
        ],
        "Sad": [
            "37i9dQZF1DX7Ocj98s7NjI",  # Bollywood Sad Songs
            "37i9dQZF1DWVlYsZJXqdhp",  # Hindi Heartbreak
        ],
        "Angry": [
            "37i9dQZF1DWYkaDif7Ztbp",  # Desi Hip Hop
            "37i9dQZF1DX7c9fUtMubKk",  # Hindi Rock
        ],
        "Calm": [
            "37i9dQZF1DWZqJzS0AcvAa",  # Peaceful Piano
            "37i9dQZF1DX0XjVJ4I6Jzw",  # Indian Classical Focus
        ],
        "Energetic": [
            "37i9dQZF1DX1clOuib1KtQ",  # Punjabi 101
            "37i9dQZF1DWYKyaJhUILyq",  # Bollywood Dance Hits
        ],
        "Romantic": [
            "37i9dQZF1DX2PQDq5P6t5a",  # Love Bollywood
            "37i9dQZF1DWV7EzJMK2FUI",  # Romantic Hindi
        ]
    }

    playlists = mood_playlists_india.get(mood, [])
    all_recommendations = []

    for playlist_id in playlists:
        try:
            tracks = sp.playlist_tracks(playlist_id, limit=20, market='IN')

            for item in tracks['items']:
                track = item['track']
                if track:
                    all_recommendations.append({
                        'id': track['id'],
                        'name': track['name'],
                        'artist': track['artists'][0]['name'],
                        'url': track['album']['images'][1]['url'] if track['album']['images'] and len(track['album']['images']) > 1 else None,
                        'popularity': track['popularity']
                    })
        except Exception as e:
            print(f"Error getting playlist {playlist_id}: {e}")

    return pd.DataFrame(all_recommendations).drop_duplicates(subset=['id'])





def test_india_recommendations():
    """Test the India-specific recommendation system"""
    moods = ["Happy", "Sad", "Angry", "Calm", "Energetic", "Romantic"]

    print("🇮🇳 TESTING INDIA-SPECIFIC MOOD RECOMMENDATIONS 🇮🇳")
    print("="*60)

    # Test 1: General Indian recommendations
    for mood in moods:
        try:
            print(f"\n--- Testing {mood} mood for India ---")
            recommendations = Recommend_Top40_By_Mood_India(mood)

            if not recommendations.empty:
                print(f"✅ Successfully found {len(recommendations)} songs for {mood} mood!")
                # Show top 3
                top_3 = recommendations.head(3)
                print("Top 3 recommendations:")
                for i, (_, row) in enumerate(top_3.iterrows(), 1):
                    print(f"  {i}. {row['name']} - {row['artist']}")
            else:
                print(f"❌ No songs found for {mood} mood in India")

        except Exception as e:
            print(f"💥 Error processing {mood} mood: {e}")



# Run the India-specific test
if __name__ == "__main__":
    test_india_recommendations()


🇮🇳 TESTING INDIA-SPECIFIC MOOD RECOMMENDATIONS 🇮🇳

--- Testing Happy mood for India ---

Searching for Happy mood songs popular in India...
Search query: bollywood dance upbeat happy hindi punjabi
🎵 Found 40 recommendations for Happy mood in India!

=== Top 10 Happy Mood Recommendations (Popular in India) ===
1. Nashe Si Chadh Gayi - Vishal-Shekhar (Popularity: 72)
2. Bom Diggy Diggy - Zack Knight (Popularity: 67)
3. Tauba Tauba - Karan Aujla (Popularity: 67)
4. Gallan Goodiyaan - Yashita Sharma (Popularity: 66)
5. Dance Basanti - Sachin-Jigar (Popularity: 65)
6. O Saki Saki (From "Batla House") - Vishal-Shekhar (Popularity: 65)
7. Dj Waley Babu (feat. Aastha Gill) - Badshah (Popularity: 65)
8. Illegal Weapon 2.0 (From "Street Dancer 3D") - Jasmine Sandlas (Popularity: 64)
9. Laung Da Lashkara - Mahalakshmi Iyer (Popularity: 63)
10. Kala Chashma - Prem & Hardeep (Popularity: 63)
✅ Successfully found 40 songs for Happy mood!
Top 3 recommendations:
  1. Nashe Si Chadh Gayi - Vishal-Shekh

ERROR:spotipy.client:HTTP Error for GET to https://api.spotify.com/v1/playlists/37i9dQZEVXbLZ52XmnySJg with Params: {'fields': None, 'market': None, 'additional_types': 'track'} returned 404 due to Resource not found


Error getting Indian top charts: http status: 404, code: -1 - https://api.spotify.com/v1/playlists/37i9dQZEVXbLZ52XmnySJg?additional_types=track:
 Resource not found, reason: None
❌ Could not fetch Indian top charts


In [ ]:
Recommend_India_By_Mood("Happy")
Recommend_India_By_Mood("Sad")



Searching Top India songs for mood: Happy


ERROR:spotipy.client:HTTP Error for GET to https://api.spotify.com/v1/playlists/37i9dQZEVXbLZ52XmnySJg/tracks with Params: {'limit': 50, 'offset': 0, 'fields': None, 'market': None, 'additional_types': 'track'} returned 404 due to Resource not found


Error fetching Top 50 India: http status: 404, code: -1 - https://api.spotify.com/v1/playlists/37i9dQZEVXbLZ52XmnySJg/tracks?limit=50&offset=0&additional_types=track:
 Resource not found, reason: None
❌ Could not fetch India Top 50 songs

Searching Top India songs for mood: Sad


ERROR:spotipy.client:HTTP Error for GET to https://api.spotify.com/v1/playlists/37i9dQZEVXbLZ52XmnySJg/tracks with Params: {'limit': 50, 'offset': 0, 'fields': None, 'market': None, 'additional_types': 'track'} returned 404 due to Resource not found


Error fetching Top 50 India: http status: 404, code: -1 - https://api.spotify.com/v1/playlists/37i9dQZEVXbLZ52XmnySJg/tracks?limit=50&offset=0&additional_types=track:
 Resource not found, reason: None
❌ Could not fetch India Top 50 songs


""


In [ ]:
mood='Sad'
try:
            # Method 1: Simple search
            print(f"\n--- Method 1: Keyword Search for {mood} ---")
            recommendations = Recommend_Top40_By_Mood_Simple(mood)

            if recommendations.empty:
                # Method 2: Curated playlists
                print(f"\n--- Method 2: Curated Playlist for {mood} ---")
                recommendations = Get_Mood_Playlists(mood)

            if not recommendations.empty:
                print(f"✅ Successfully found {len(recommendations)} songs for {mood} mood!")
                if len(recommendations) > 0:
                    visualize_songs(recommendations.head(10))
            else:
                print(f"❌ Failed to find songs for {mood} mood")

except Exception as e:
            print(f"💥 Error processing {mood} mood: {e}")

print("\n" + "-"*50 + "\n")



--- Method 1: Keyword Search for Sad ---

Searching for Sad mood...
Search query: sad acoustic emotional melancholic
🎵 Found 40 recommendations for Sad mood!

=== Top 10 Sad Mood Recommendations ===
1. Cose della vita (Can't Stop Thinking of You) (feat. Tina Turner) - Eros Ramazzotti (Popularity: 59)
2. Cose Della Vita (with Tina Turner) - Eros Ramazzotti (Popularity: 58)
3. I'm Back - girl in red (Popularity: 51)
4. Melancholy - Jonas Sandberg (Popularity: 47)
5. Sad Emotional Piano - DS Productions (Popularity: 42)
6. Emotional Piano - OlexandrMusic (Popularity: 40)
7. Sad Emotional Violin Music - Ecveezer (Popularity: 39)
8. Sad Emotional Piano - Slowed Down - DS Productions (Popularity: 36)
9. Late Night Melancholy (Piano) - Rude Boy (Popularity: 36)
10. Ambient Sad Piano - DS Productions (Popularity: 36)
✅ Successfully found 40 songs for Sad mood!
💥 Error processing Sad mood: Number of rows must be a positive integer, not 3.0

--------------------------------------------------



<Figure size 1500x600 with 0 Axes>

In [ ]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Initialize Spotify client (you'll need to set your credentials)
# sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(
#     client_id="your_client_id",
#     client_secret="your_client_secret"
# ))

# ==============================
# Mood → Search Query Mapping
# ==============================
def ChooseDataset(mood):
    """
    Map mood to Spotify search parameters
    """
    mood_to_search = {
        "Disgust": "rock metal angry intense",
        "Angry": "metal hard rock aggressive",
        "Fear": "ambient dark atmospheric scary",
        "Happy": "pop dance upbeat happy",
        "Sad": "sad acoustic emotional melancholic",
        "Surprise": "electronic experimental surprising",
        "Calm": "chill ambient peaceful calm",
        "Energetic": "edm energetic workout pump-up"
    }
    return mood_to_search.get(mood, "pop")


# ==============================
# Simple Keyword Search (FIXED)
# ==============================
def Search_Spotify_By_Mood_Simple(mood_query, limit=40):
    """
    Simple search using mood keywords without audio features
    """
    try:
        results = sp.search(q=mood_query, type='track', limit=limit)

        recommendations = []
        for track in results['tracks']['items']:
            # Added safety checks for missing data
            artist_name = track['artists'][0]['name'] if track['artists'] else "Unknown Artist"
            album_name = track['album']['name'] if track['album'] else "Unknown Album"

            # Better image URL handling
            image_url = None
            if track['album']['images']:
                if len(track['album']['images']) > 1:
                    image_url = track['album']['images'][1]['url']  # Medium size
                elif len(track['album']['images']) > 0:
                    image_url = track['album']['images'][0]['url']  # Fallback to first available

            recommendations.append({
                'id': track['id'],
                'name': track['name'],
                'artist': artist_name,
                'album': album_name,
                'url': image_url,
                'preview_url': track['preview_url'],
                'external_url': track['external_urls']['spotify'],
                'popularity': track['popularity']
            })

        return pd.DataFrame(recommendations)

    except Exception as e:
        print(f"Error in simple search: {e}")
        return pd.DataFrame()


# ==============================
# Genre-Based Search (IMPROVED)
# ==============================
def Search_Spotify_By_Genre(genre, limit=40):
    """
    Search by genre with better error handling
    """
    try:
        results = sp.search(q=f'genre:"{genre}"', type='track', limit=limit)

        recommendations = []
        for track in results['tracks']['items']:
            artist_name = track['artists'][0]['name'] if track['artists'] else "Unknown Artist"

            image_url = None
            if track['album']['images']:
                if len(track['album']['images']) > 1:
                    image_url = track['album']['images'][1]['url']
                elif len(track['album']['images']) > 0:
                    image_url = track['album']['images'][0]['url']

            recommendations.append({
                'id': track['id'],
                'name': track['name'],
                'artist': artist_name,
                'url': image_url,
                'popularity': track['popularity']
            })

        return pd.DataFrame(recommendations)

    except Exception as e:
        print(f"Error searching by genre: {e}")
        return pd.DataFrame()


# ==============================
# Playlist-Based Recommendations (FIXED)
# ==============================
def Get_User_Playlist_Tracks(playlist_id, limit=50):
    """
    Fetch track IDs from a user's playlist with better error handling
    """
    try:
        # Handle both full URI and just ID
        if playlist_id.startswith('spotify:playlist:'):
            playlist_id = playlist_id.split(':')[-1]
        elif playlist_id.startswith('https://open.spotify.com/playlist/'):
            playlist_id = playlist_id.split('/')[-1].split('?')[0]

        tracks = sp.playlist_tracks(playlist_id, limit=limit)
        track_ids = []

        for item in tracks['items']:
            if item and item['track'] and item['track']['id']:
                track_ids.append(item['track']['id'])

        return track_ids
    except Exception as e:
        print(f"Error getting user playlist: {e}")
        return []


def Get_Similar_To_Playlist(playlist_id, mood=None, limit=40):
    """
    🔧 FIXED: Recommend songs similar to the user's playlist
    The main issue was seed_tracks being passed as a string instead of a list
    """
    seed_tracks = Get_User_Playlist_Tracks(playlist_id, limit=10)
    if not seed_tracks:
        print("❌ No seed tracks found in playlist.")
        return pd.DataFrame()

    # Take max 5 seeds (Spotify API limit)
    seed_tracks = seed_tracks[:5]
    print(f"🎯 Using {len(seed_tracks)} seed tracks: {seed_tracks}")

    try:
        # Mood filters (Spotify audio features)
        mood_filters = {
            "Happy": {"target_valence": 0.8, "target_energy": 0.7},
            "Sad": {"target_valence": 0.2, "target_acousticness": 0.6},
            "Calm": {"target_energy": 0.3, "target_instrumentalness": 0.5},
            "Energetic": {"target_energy": 0.9, "target_danceability": 0.8},
            "Angry": {"target_energy": 0.9, "target_valence": 0.3},
            "Fear": {"target_energy": 0.4, "target_valence": 0.2, "target_acousticness": 0.7},
            "Surprise": {"target_energy": 0.6, "target_danceability": 0.6},
            "Disgust": {"target_energy": 0.8, "target_valence": 0.2}
        }
        filters = mood_filters.get(mood, {})

        # 🔧 CRITICAL FIX: Pass seed_tracks as a list, not joined string
        results = sp.recommendations(
            seed_tracks=seed_tracks,  # ✅ This is already a list
            limit=limit,
            **filters
        )

        recommendations = []
        for track in results['tracks']:
            artist_name = track['artists'][0]['name'] if track['artists'] else "Unknown Artist"
            album_name = track['album']['name'] if track['album'] else "Unknown Album"

            image_url = None
            if track['album']['images']:
                if len(track['album']['images']) > 1:
                    image_url = track['album']['images'][1]['url']
                elif len(track['album']['images']) > 0:
                    image_url = track['album']['images'][0]['url']

            recommendations.append({
                'id': track['id'],
                'name': track['name'],
                'artist': artist_name,
                'album': album_name,
                'url': image_url,
                'preview_url': track['preview_url'],
                'external_url': track['external_urls']['spotify'],
                'popularity': track['popularity']
            })

        return pd.DataFrame(recommendations)

    except Exception as e:
        print(f"❌ Error generating playlist-based recommendations: {e}")
        print(f"🔍 Seeds used: {seed_tracks}")
        print(f"🔍 Filters used: {filters}")
        return pd.DataFrame()


# ==============================
# Advanced Recommendation with Fallbacks
# ==============================
def Get_Recommendations_By_Seeds_And_Genres(seed_tracks=None, seed_genres=None, mood=None, limit=40):
    """
    Advanced recommendation using both seed tracks and genres
    """
    try:
        # Mood filters
        mood_filters = {
            "Happy": {"target_valence": 0.8, "target_energy": 0.7},
            "Sad": {"target_valence": 0.2, "target_acousticness": 0.6},
            "Calm": {"target_energy": 0.3, "target_instrumentalness": 0.5},
            "Energetic": {"target_energy": 0.9, "target_danceability": 0.8},
            "Angry": {"target_energy": 0.9, "target_valence": 0.3},
        }
        filters = mood_filters.get(mood, {})

        # Prepare parameters
        params = {"limit": limit, **filters}

        if seed_tracks and len(seed_tracks) > 0:
            params["seed_tracks"] = seed_tracks[:5]  # Max 5 seeds

        if seed_genres and len(seed_genres) > 0:
            params["seed_genres"] = seed_genres[:5]  # Max 5 seeds

        # Ensure we have at least one seed
        if not seed_tracks and not seed_genres:
            # Default genres based on mood
            default_genres = {
                "Happy": ["pop", "dance"],
                "Sad": ["acoustic", "indie"],
                "Energetic": ["electronic", "dance"],
                "Angry": ["metal", "rock"],
                "Calm": ["ambient", "chill"]
            }
            params["seed_genres"] = default_genres.get(mood, ["pop"])

        results = sp.recommendations(**params)

        recommendations = []
        for track in results['tracks']:
            recommendations.append({
                'id': track['id'],
                'name': track['name'],
                'artist': track['artists'][0]['name'] if track['artists'] else "Unknown",
                'album': track['album']['name'] if track['album'] else "Unknown",
                'url': track['album']['images'][1]['url'] if track['album']['images'] and len(track['album']['images']) > 1 else None,
                'preview_url': track['preview_url'],
                'external_url': track['external_urls']['spotify'],
                'popularity': track['popularity']
            })

        return pd.DataFrame(recommendations)

    except Exception as e:
        print(f"Error in advanced recommendations: {e}")
        return pd.DataFrame()


# ==============================
# Curated Mood Playlists (IMPROVED)
# ==============================
def Get_Mood_Playlists(mood):
    """
    Get mood-based curated playlists with better error handling
    """
    # Updated playlist IDs (extract just the ID part)
    mood_playlists = {
        "Happy": "37i9dQZF1DXdPec7aLTmlC",  # Happy Hits
        "Sad": "37i9dQZF1DX7qK8ma5wgG1",   # Sad Songs
        "Angry": "37i9dQZF1DX4sWSpwq3LiO", # Rock Classics
        "Calm": "37i9dQZF1DX4WYpdgoIcn6",  # Chill Hits
        "Energetic": "37i9dQZF1DX0XUsuxWHRQd", # RapCaviar
        "Workout": "37i9dQZF1DX76Wlfdnj7AP" # Beast Mode
    }

    playlist_id = mood_playlists.get(mood)
    if not playlist_id:
        print(f"❌ No curated playlist found for mood: {mood}")
        return pd.DataFrame()

    try:
        tracks = sp.playlist_tracks(playlist_id, limit=40)
        recommendations = []

        for item in tracks['items']:
            if not item or not item['track']:
                continue

            track = item['track']
            artist_name = track['artists'][0]['name'] if track['artists'] else "Unknown Artist"

            image_url = None
            if track['album']['images']:
                if len(track['album']['images']) > 1:
                    image_url = track['album']['images'][1]['url']
                elif len(track['album']['images']) > 0:
                    image_url = track['album']['images'][0]['url']

            recommendations.append({
                'id': track['id'],
                'name': track['name'],
                'artist': artist_name,
                'url': image_url,
                'popularity': track['popularity'],
                'external_url': track['external_urls']['spotify']
            })

        return pd.DataFrame(recommendations)

    except Exception as e:
        print(f"Error getting curated playlist for {mood}: {e}")
        return pd.DataFrame()


# ==============================
# Smart Recommendation Function
# ==============================
def Smart_Mood_Recommendations(mood, user_playlist_id=None, limit=40):
    """
    🧠 Intelligent recommendation system with multiple fallback strategies
    """
    print(f"\n🎵 Getting recommendations for {mood} mood...")

    recommendations = pd.DataFrame()

    # Strategy 1: User playlist-based (if provided)
    if user_playlist_id:
        print("📱 Trying playlist-based recommendations...")
        recommendations = Get_Similar_To_Playlist(user_playlist_id, mood=mood, limit=limit)

        if not recommendations.empty:
            print(f"✅ Found {len(recommendations)} playlist-based recommendations!")
            return recommendations

    # Strategy 2: Curated mood playlists
    print("🎯 Trying curated mood playlists...")
    recommendations = Get_Mood_Playlists(mood)

    if not recommendations.empty:
        print(f"✅ Found {len(recommendations)} songs from curated playlists!")
        return recommendations.head(limit)

    # Strategy 3: Keyword search
    print("🔍 Trying keyword search...")
    mood_query = ChooseDataset(mood)
    recommendations = Search_Spotify_By_Mood_Simple(mood_query, limit=limit)

    if not recommendations.empty:
        print(f"✅ Found {len(recommendations)} songs via keyword search!")
        return recommendations

    # Strategy 4: Genre-based fallback
    print("🎸 Trying genre-based search...")
    genre_map = {
        "Happy": "pop",
        "Sad": "acoustic",
        "Angry": "metal",
        "Energetic": "dance",
        "Calm": "ambient"
    }
    genre = genre_map.get(mood, "pop")
    recommendations = Search_Spotify_By_Genre(genre, limit=limit)

    if not recommendations.empty:
        print(f"✅ Found {len(recommendations)} songs via genre search!")
        return recommendations

    print("❌ No recommendations found with any method")
    return pd.DataFrame()


# ==============================
# Display Functions
# ==============================
def Display_Recommendations(recommendations, mood, top_n=10):
    """
    Display recommendations in a nice format
    """
    if recommendations.empty:
        print(f"❌ No recommendations found for {mood} mood")
        return

    # Sort by popularity
    recommendations = recommendations.sort_values('popularity', ascending=False)

    print(f"\n🎵 Top {top_n} {mood} Mood Recommendations:")
    print("="*60)

    for i, (_, row) in enumerate(recommendations.head(top_n).iterrows(), 1):
        popularity = row.get('popularity', 0)
        preview = "🎧" if row.get('preview_url') else "🔇"

        print(f"{i:2d}. {preview} {row['name']}")
        print(f"     👤 {row['artist']} | 📊 {popularity}/100")

        if 'external_url' in row and row['external_url']:
            print(f"     🔗 {row['external_url']}")
        print()


# ==============================
# Master Test Function (IMPROVED)
# ==============================
def test_recommendations():
    """Test all recommendation methods with better error handling"""
    moods = ["Happy", "Sad", "Angry", "Calm", "Energetic"]

    # Replace with a real playlist ID for testing
    user_playlist_id = "6kNnPvI9hedEDyNbJApstX"  # Just the ID part

    for mood in moods:
        try:
            print(f"\n{'='*60}")
            print(f"🧪 Testing {mood.upper()} mood recommendations")
            print(f"{'='*60}")

            # Use the smart recommendation system
            recommendations = Smart_Mood_Recommendations(
                mood=mood,
                user_playlist_id=user_playlist_id,
                limit=40
            )

            if not recommendations.empty:
                Display_Recommendations(recommendations, mood, top_n=10)
            else:
                print(f"❌ Failed to find any recommendations for {mood}")

        except Exception as e:
            print(f"💥 Critical error processing {mood}: {e}")

        print("\n" + "-"*60)


# ==============================
# Quick Test Function
# ==============================
def quick_test(mood="Happy"):
    """Quick test for a single mood"""
    print(f"🚀 Quick test for {mood} mood...")

    recommendations = Smart_Mood_Recommendations(mood)
    Display_Recommendations(recommendations, mood)

    return recommendations


# ==============================
# Usage Examples
# ==============================
if __name__ == "__main__":
    # Uncomment to run tests
    test_recommendations()

    # Quick test
    # recs = quick_test("Energetic")

    print("🎵 Spotify Music Recommendation System Ready!")
    print("💡 Make sure to set up your Spotify API credentials first:")
    print("   sp = spotipy.Spotify(client_credentials_manager=...)")

ERROR:spotipy.client:HTTP Error for GET to https://api.spotify.com/v1/recommendations with Params: {'limit': 40, 'seed_tracks': '35RWttZt8zGGzT0mWtlXpn,5XD2tP1DRH2NJgp45HiEQA,0rMeOAvbQZ2RTG4B6L1M4p,1OEmd2iXU8pWG7CrKNpGUg,5O2P9iiztwhomNh8xkR9lJ', 'target_energy': 0.7, 'target_valence': 0.8} returned 404 due to None



🧪 Testing HAPPY mood recommendations

🎵 Getting recommendations for Happy mood...
📱 Trying playlist-based recommendations...
🎯 Using 5 seed tracks: ['35RWttZt8zGGzT0mWtlXpn', '5XD2tP1DRH2NJgp45HiEQA', '0rMeOAvbQZ2RTG4B6L1M4p', '1OEmd2iXU8pWG7CrKNpGUg', '5O2P9iiztwhomNh8xkR9lJ']
❌ Error generating playlist-based recommendations: http status: 404, code: -1 - https://api.spotify.com/v1/recommendations?limit=40&seed_tracks=35RWttZt8zGGzT0mWtlXpn%2C5XD2tP1DRH2NJgp45HiEQA%2C0rMeOAvbQZ2RTG4B6L1M4p%2C1OEmd2iXU8pWG7CrKNpGUg%2C5O2P9iiztwhomNh8xkR9lJ&target_energy=0.7&target_valence=0.8:
 None, reason: None
🔍 Seeds used: ['35RWttZt8zGGzT0mWtlXpn', '5XD2tP1DRH2NJgp45HiEQA', '0rMeOAvbQZ2RTG4B6L1M4p', '1OEmd2iXU8pWG7CrKNpGUg', '5O2P9iiztwhomNh8xkR9lJ']
🔍 Filters used: {'target_valence': 0.8, 'target_energy': 0.7}
🎯 Trying curated mood playlists...


ERROR:spotipy.client:HTTP Error for GET to https://api.spotify.com/v1/playlists/37i9dQZF1DXdPec7aLTmlC/tracks with Params: {'limit': 40, 'offset': 0, 'fields': None, 'market': None, 'additional_types': 'track'} returned 404 due to Resource not found


Error getting curated playlist for Happy: http status: 404, code: -1 - https://api.spotify.com/v1/playlists/37i9dQZF1DXdPec7aLTmlC/tracks?limit=40&offset=0&additional_types=track:
 Resource not found, reason: None
🔍 Trying keyword search...


ERROR:spotipy.client:HTTP Error for GET to https://api.spotify.com/v1/recommendations with Params: {'limit': 40, 'seed_tracks': '35RWttZt8zGGzT0mWtlXpn,5XD2tP1DRH2NJgp45HiEQA,0rMeOAvbQZ2RTG4B6L1M4p,1OEmd2iXU8pWG7CrKNpGUg,5O2P9iiztwhomNh8xkR9lJ', 'target_acousticness': 0.6, 'target_valence': 0.2} returned 404 due to None


...

Error getting curated playlist for Calm: http status: 404, code: -1 - https://api.spotify.com/v1/playlists/37i9dQZF1DX4WYpdgoIcn6/tracks?limit=40&offset=0&additional_types=track:
 Resource not found, reason: None
🔍 Trying keyword search...


ERROR:spotipy.client:HTTP Error for GET to https://api.spotify.com/v1/recommendations with Params: {'limit': 40, 'seed_tracks': '35RWttZt8zGGzT0mWtlXpn,5XD2tP1DRH2NJgp45HiEQA,0rMeOAvbQZ2RTG4B6L1M4p,1OEmd2iXU8pWG7CrKNpGUg,5O2P9iiztwhomNh8xkR9lJ', 'target_danceability': 0.8, 'target_energy': 0.9} returned 404 due to None


✅ Found 40 songs via keyword search!

🎵 Top 10 Calm Mood Recommendations:
 1. 🔇 God's Plan
     👤 Drake | 📊 84/100
     🔗 https://open.spotify.com/track/6DCZcSspjsKoFjzjrWoCdn

 2. 🔇 Celebrate (with Offset)
     👤 Juice WRLD | 📊 59/100
     🔗 https://open.spotify.com/track/6hEzT3fE6ecwWvhlqiFCLF

 3. 🔇 ON TOP 2
     👤 Karan Aujla | 📊 50/100
     🔗 https://open.spotify.com/track/2G4GimpAMeWaZ6VkcQ1RT9

 4. 🔇 Ambient Forest and Rain
     👤 chill phil | 📊 33/100
     🔗 https://open.spotify.com/track/3e0BTU443nQRbgKiiNjUGp

 5. 🔇 Ambient Piano Meditation Music for Restful Sleep
     👤 Quiet Mind Relax | 📊 30/100
     🔗 https://open.spotify.com/track/7epv5ZjPcVVzLkb4bPj4F5

 6. 🔇 Soft Ambient Tones for Late Night Calm
     👤 The Namaste | 📊 29/100
     🔗 https://open.spotify.com/track/0AQQtCLRwqWJI58gfYecNp

 7. 🔇 Ambient Peace
     👤 Relaxing Music for Stress Relief | 📊 26/100
     🔗 https://open.spotify.com/track/1sk135q5lXzW014oN36TuJ

 8. 🔇 Ambient Study Music
     👤 Study Music Project

ERROR:spotipy.client:HTTP Error for GET to https://api.spotify.com/v1/playlists/37i9dQZF1DX0XUsuxWHRQd/tracks with Params: {'limit': 40, 'offset': 0, 'fields': None, 'market': None, 'additional_types': 'track'} returned 404 due to Resource not found


Error getting curated playlist for Energetic: http status: 404, code: -1 - https://api.spotify.com/v1/playlists/37i9dQZF1DX0XUsuxWHRQd/tracks?limit=40&offset=0&additional_types=track:
 Resource not found, reason: None
🔍 Trying keyword search...
✅ Found 40 songs via keyword search!

🎵 Top 10 Energetic Mood Recommendations:
 1. 🔇 Pump It
     👤 Electric Callboy | 📊 63/100
     🔗 https://open.spotify.com/track/6emZMVNvIxW57fhxPlyxLp

 2. 🔇 The New Workout Plan
     👤 Kanye West | 📊 62/100
     🔗 https://open.spotify.com/track/1Vp4St7JcXaUoJcIahtf3L

 3. 🔇 Pump It
     👤 Electric Callboy | 📊 50/100
     🔗 https://open.spotify.com/track/3iXNlPQNYPrtimAEM49PsG

 4. 🔇 Last Night EDM (Workout Pop Mix)
     👤 Dhrriti Saharan | 📊 37/100
     🔗 https://open.spotify.com/track/38901bdbC0aZQ6G76IPSJn

 5. 🔇 EDM DROP TRANCE
     👤 Dj Swam | 📊 33/100
     🔗 https://open.spotify.com/track/7gEKgRBViOmK4b2CHMoxBk

 6. 🔇 Dhoom Again - Workout Remix
     👤 Vishal Dadlani | 📊 23/100
     🔗 https://open.spot

In [ ]:
seeds = [
    "35RWttZt8zGGzT0mWtlXpn",
    "5XD2tP1DRH2NJgp45HiEQA",
    "0rMeOAvbQZ2RTG4B6L1M4p"
]

results = sp.recommendations(
    seed_tracks=["35RWttZt8zGGzT0mWtlXpn"],  # just one
    limit=10
)


for track in results['tracks']:
    print(track['name'], "-", track['artists'][0]['name'])



ERROR:spotipy.client:HTTP Error for GET to https://api.spotify.com/v1/recommendations with Params: {'limit': 10, 'seed_tracks': '35RWttZt8zGGzT0mWtlXpn'} returned 404 due to None


SpotifyException: http status: 404, code: -1 - https://api.spotify.com/v1/recommendations?limit=10&seed_tracks=35RWttZt8zGGzT0mWtlXpn:
 None, reason: None